# Command Line version: 



## Tissue segmentation
mrconvert T1.nii T1.mif

5ttgen fsl T1.mif 5tt_nocoreg.mif

mrconvert 5tt_nocoreg.mif  5tt_nocoreg.nii.gz

fslroi 5tt_nocoreg.nii.gz 5tt_vol0.nii.gz 0 1

flirt -in b0.nii -ref 5tt_vol0.nii.gz -interp nearestneighbour -dof 6 -omat diff2struct_fsl.mat

transformconvert diff2struct_fsl.mat b0.nii 5tt_nocoreg.nii.gz flirt_import diff2struct_mrtrix.txt

mrtransform 5tt_nocoreg.mif -linear diff2struct_mrtrix.txt -inverse 5tt_coreg.mif 

5tt2gmwmi 5tt_coreg.mif gmwmSeed_coreg.mif

## fod generation tournier (mainly for single shell) or dhollander (single or multi-shell data and for multiple tissue types)
dwi2response dhollander dti_den_preproc_unbiased.mif wm.txt gm.txt csf.txt -voxels voxels.mif 

shview wm.txt

dwi2fod msmt_csd dti_den_preproc_unbiased.mif -mask mask.mif wm.txt wmfod.mif gm.txt gmfod.mif csf.txt csffod.mif 

mrconvert -coord 3 0 wmfod.mif - | mrcat csffod.mif gmfod.mif vf.mif

mrview vf.mif -odf.load_sh wmfod.mif

mtnormalise wmfod.mif wmfod_norm.mif gmfod.mif gmfod_norm.mif csffod.mif csffod_norm.mif -mask mask.mif 

mrview vf.mif -odf.load_sh wmfod_norm.mif

## tractogram

tckgen -act 5tt_coreg.mif -backtrack -seed_gmwmi gmwmSeed_coreg.mif -nthreads 8 -maxlength 250 -cutoff 0.06 -select 10000000 wmfod_norm.mif tracks_10M.tck

tckedit tracks_10M.tck -number 200k smallerTracks_200k.tck

tcksift2 -act 5tt_coreg.mif -out_mu sift_mu.txt -out_coeffs sift_coeffs.txt -nthreads 8 tracks_10M.tck wmfod_norm.mif sift_1M.txt

In [ ]:
import os
os.environ['PATH'] += ':/Users/christianraimondo/ants-2.5.0/bin'
nthreads = os.environ.get('MRTRIX_NTHREADS', 'Not set')
print(f'MRTRIX_NTHREADS is {nthreads}')

import tempfile
import subprocess
from tqdm import tqdm
import numpy as np
import nibabel as nib
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from dipy.reconst.dti import gradient_table, TensorModel
from dipy.reconst.dti import fractional_anisotropy, mean_diffusivity, color_fa
from dipy.core.gradients import gradient_table
import nibabel as nib
from dipy.reconst.dti import TensorModel, mean_diffusivity, decompose_tensor
import shutil   
from dipy.io import read_bvals_bvecs


def run_command(command, step_name):
    print(f"Running: {step_name}")
    result = subprocess.run(command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        print(f"Command failed with code {result.returncode}: {result.stderr}")
    return result


def tracto(patient_dir):
    steps = ["Converting T1 image", "Generating 5-tissue-type segmented tissue image", "Converting 5-tissue image", "Extracting ROI", "Registering diffusion image to T1", "Converting linear transformation matrices", "Transforming image", "Generating mask for streamline seeding", "Estimating response functions for spherical deconvolution", "Estimating fibre orientation distributions using spherical deconvolution", "Concatenating images", "Performing log-domain intensity normalization", "Generating tractogram", "Reducing number of tracts", "Optimizing per-streamline cross-section multipliers"]
    
    dti_dir = os.path.join(patient_dir,"dti_outputs" )
    t1_dir = os.path.join(patient_dir, "struct")
    output_dir = os.path.join(patient_dir, "tractography_outputs")
    os.makedirs(output_dir, exist_ok=True)


    for i, step_name in enumerate(tqdm(steps, desc="Processing Tractography", unit="step")):

        #tissue segmentation: anatomically constrained tractography to improve streamline accuracy
        if step_name == "Converting T1 image":
            run_command(f"mrconvert {t1_dir}/t1.nii {t1_dir}/t1.mif", step_name)
        
        elif step_name == "Generating 5-tissue type segmented tissue image":
            run_command(f"5ttgen fsl {t1_dir}/t1.mif {t1_dir}/5tt_nocoreg.mif", step_name)

        elif step_name == "Converting 5-tissue image":
            run_command(f"mrconvert {t1_dir}/5tt_nocoreg.mif {t1_dir}/5tt_nocoreg.nii.gz", step_name)
        
        elif step_name == "Extracting ROI":
            run_command(f"fslroi {t1_dir}/5tt_nocoreg.nii.gz {t1_dir}/5tt_vol0.nii.gz 0 1", step_name)
        
        elif step_name == "Registering diffusion image to T1":
            run_command(f"flirt -in {dti_dir}/b0.nii -ref {t1_dir}/5tt_vol0.nii.gz -interp nearestneighbour -dof 6 -omat {t1_dir}/diff2struct_fsl.mat", step_name)

        elif step_name == "Converting linear transformation matrices":
            run_command(f"transformconvert {t1_dir}/diff2struct_fsl.mat {dti_dir}/b0.nii {t1_dir}/5tt_nocoreg.nii.gz flirt_import {t1_dir}/diff2struct_mrtrix.txt", step_name)

        elif step_name == "Transforming image":
            run_command(f"mrtransform {t1_dir}/5tt_nocoreg.mif -linear {t1_dir}/diff2struct_mrtrix.txt -inverse {t1_dir}/5tt_coreg.mif", step_name)

        elif step_name == "Generating mask for streamline seeding":
            run_command(f"5tt2gmwmi {t1_dir}/5tt_coreg.mif {t1_dir}/gmwmSeed_coreg.mif", step_name)

        #multi-shell multi-tissue fibre orientation distribution calculation
        elif step_name == "Estimating response functions for spherical deconvolution":
            run_command(f"dwi2response dhollander {dti_dir}/dti_denoise_preproc_unbiased.mif {output_dir}/wm.txt {output_dir}/gm.txt {output_dir}/csf.txt -voxels {output_dir}/voxels.mif", step_name)

        elif step_name == "Estimating fibre orientation distributions using spherical deconvolution":
            run_command(f"dwi2fod msmt_csd {dti_dir}/dti_denoise_preproc_unbiased.mif -mask {dti_dir}/mask.mif {output_dir}/wm.txt {output_dir}/wmfod.mif {output_dir}/gm.txt {output_dir}/gmfod.mif {output_dir}/csf.txt {output_dir}/csffod.mif", step_name)
        
        elif step_name == "Concatenating images":
            run_command(f"mrconvert -coord 3 0 {output_dir}/wmfod.mif - | mrcat {output_dir}/csffod.mif {output_dir}/gmfod.mif {output_dir}/vf.mif", step_name)

        elif step_name == "Performing log-domain intensity normalization":
            run_command(f"mtnormalise {output_dir}/wmfod.mif {output_dir}/wmfod_norm.mif {output_dir}/gmfod.mif {output_dir}/gmfod_norm.mif {output_dir}/csffod.mif {output_dir}/csffod_norm.mif -mask {dti_dir}/mask.mif", step_name)

        #tractogram generation
        elif step_name == "Generating tractogram":
            run_command(f"tckgen -act {t1_dir}/5tt_coreg.mif -backtrack -seed_gmwmi {t1_dir}/gmwmSeed_coreg.mif -nthreads 8 -maxlength 250 -cutoff 0.06 -select 10000000 {output_dir}/wmfod_norm.mif {output_dir}/tracks_10M.tck", step_name)
        
        elif step_name == "Reducing number of tracts":
            run_command(f"tckedit {output_dir}/tracks_10M.tck -number 200k {output_dir}/smallerTracks_200k.tck", step_name)

        elif step_name == "Optimizing per-streamline cross-section multipliers":
            run_command(f"tcksift2 -act {t1_dir}/5tt_coreg.mif -out_mu {output_dir}/sift_mu.txt -out_coeffs {output_dir}/sift_coeffs.txt -nthreads 8 {output_dir}/tracks_10M.tck wmfod_norm.mif {output_dir}/sift_1M.txt", step_name)

#this is the folder with all of the pt folders in it
base_path = ""

patient_list = os.listdir(base_path)

for patient in patient_list
    patient_dir = os.path.join(base_path, patient)
    try:
        print(f"starting processing of {patient}.")
        tracto(patient_dir)
        print(f"{patient_dir} processing completed successfully.")
    except Exception as e:
        print(f"Error while processing {patient}: {str(e)}")